# Sequence-to-Sequence CNN-BiLSTM Based Glottal Closure Instant Detection from Raw Speech

This is an example of a Python code to train and test a CNN-BiLSTM model, a joint convolutional (CNN) and recurrent (RNN) neural network model, for detecting glottal closure instants (GCIs) in the speech signal. See the [corresponding paper](paper/Matousek_ANNPR2022_paper.pdf) for more details.

[Keras](https://keras.io/) (v2.6.0) with [TensorFlow](https://www.tensorflow.org/) (v2.6.0) backend are used to train and evaluate the CNN-BiLSTM model.

Prerequisities are stored in the [requirements](requirements.txt) file.

Firstly, we import libraries.

In [1]:
import os
import os.path as osp
import numpy as np
import random as pyrandom
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.models import model_from_json
import librosa as lr
import logging
import utils as ut

## Data

To show the training and evaluation of the CNN-BiLSTM model, we describe data firstly. Note that just a [sample of data](data/sample) will be used in this tutorial (only 40 waveforms for training and 2 waveforms for testing from 2 voice talents). In the [corresponding paper](paper/Matousek_ANNPR2022_paper.pdf), 3200 waveforms from 16 voice talents were used.

The following sample of data is used:
* `spc ...` speech waveforms sampled at 16 kHz
* `gci ...` ground truth GCIs

We used the [Multi-Phase Algorithm](http://www.sciencedirect.com/science/article/pii/S0167639311000094) (MPA) to detect GCIs from the contemporaneous electroglottograph (EGG) signal and used the detected GCIs as the ground truth ones.

The ground truth GCIs are available in the [wavesurfer](http://www.speech.kth.se/wavesurfer) format

```
0.234687 0.234687 V
0.242312 0.242312 V
0.250250 0.250250 V
0.258062 0.258062 V
0.265937 0.265937 V
```

## Settings

The following code sets the randomness and tries to ensure reproducibility

In [2]:
# Seed value
SEED = 8
# Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['CUDA_VISIBLE_DEVICES'] = ''
# Set python built-in pseudo-random generator at a fixed value
pyrandom.seed(SEED)
# Set numpy pseudo-random generator at a fixed value
np.random.seed(SEED)
# Set the tensorflow pseudo-random generator at a fixed value
tf.random.set_seed(SEED)
# Suppress Tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Logging
logging.basicConfig(level=logging.INFO)

Next, we set up various (hyper-)parameters. Note that for this demonstration we use a simple model

In [3]:
# Data
spc_dir = 'data/spc'         # directory with speech waveforms
pm_dir = 'data/gci'          # directory with ground truth GCIs
trainlist = 'data/train.txt' # list of training utterances
vallist = 'data/val.txt'     # list of validation utterances

# Windowing
freq_samp = 16000  # sampling frequency of the signal to detect in
win_len = 0.010    # window length (sec)
hop_len = 0.002    # hop length (sec)
window = 'boxcar'  # 'boxcar' is equivalent to rectangular window
n_timesteps = 900  # number of time steps
pad_value = 0      # value to pad shodrter signals

# RNN layers
rnn_cells = (64,) # in the paper we use (256, 256, 256)
dropout = (0.5,)  # in the paper we use (0.5, 0.5, 0.5)
batch_norm = True # whether to use batch normalization
rnn = 'lstm'      # units used in the RNN architecture ('lstm' or 'gru')

# CNN layers
n_convs = 1            # No. of repeating convolutional layers in a single CNN block (typically 1-2) - in the paper we use 2
filters = (4,)         # No. of filters corresponds to No. of convolutional blocks - in the paper we use (16, 32, 64)
kernel_size = (3,)     # Kernel size for each convolutional block (must be the same as the number of filters)
strides = (1,)         # in the paper we use (1, 1, 1)
pool_size = (2,)       # in the paper we use (1, 1, 1)
padding = ('same',)    # in the paper we use ('same', 'same', 'same')

# Training
batch_size = 64            # in the paper we use 16
n_epochs = 4               # in the paper we use 100
optimizer = 'adam'         # optimizer
learning_rate = 0.001      # in the paper we use 0.0005
early_stop_patience = None # in the paper we use 10

# Set up measures for GCI detection
abs_dist = 0.00025    # distance threshold (msec) to detect shifted GCIs (used for IDA - identification accuracy error) 
rel_dist = 10         # distance threshold (integer percentage within current T0) to detect shifted GCIs
min_t0 = 0.02         # minimum T0 for relative-distance based comparison
sync = (0.002, 0.002) # syncing interval to search for minimum negative sample in the detected speech frame

## Training the CNN-BiLSTM model

Firsly, we load train/validation data

In [4]:
# Get train/dev utterance lists
trainset = np.loadtxt(trainlist, dtype=str).tolist()
valset = np.loadtxt(vallist, dtype=str).tolist()
# Window length in samples
n_wsamples = lr.time_to_samples(win_len, freq_samp)

# Load train data
X_train, Y_train, _ = ut.prep4input(trainset, spc_dir, pm_dir, freq_samp, n_timesteps, fft_len=win_len, hop_len=hop_len,
                                    window=window, pad_value=pad_value)
# Load validation data
X_val, Y_val, _ = ut.prep4input(valset, spc_dir, pm_dir, freq_samp, n_timesteps, fft_len=win_len, hop_len=hop_len,
                                window=window, pad_value=pad_value)

and check the shape of inputted data

In [5]:
print('Training data shape       :', X_train.shape, Y_train.shape)
print('Validation data shape     :', X_val.shape, Y_val.shape)
print('# training targets (1/0)  :  {} ({} / {})'.format(Y_train.size, len(Y_train[Y_train==1]), len(Y_train[Y_train==0])))
print('# validation targets (1/0): {} ({} / {})'.format(Y_val.size, len(Y_val[Y_val==1]), len(Y_val[Y_val==0])))
print('# samples per window      :', n_wsamples)

Training data shape       : (95, 900, 160, 1) (95, 900, 1)
Validation data shape     : (4, 900, 160, 1) (4, 900, 1)
# training targets (1/0)  :  85500 (8701 / 76799)
# validation targets (1/0): 3600 (377 / 3223)
# samples per window      : 160


In this example, we use a joint CNN-BiLSTM model which is shown in the paper to achieve the best results. The model is defined by `utils.create_model` function. A simplified scheme of the proposed CNN-BiLSTM based GCI detection is shown here:

![GCI detection](figs/gci_detection.png "CNN-BiLSTM based GCI detection model")

The CNN block (dotted line) works as a feature extractor. When omitted, "only" RNN-based GCi detection on raw speech is performed. For 16kHz input speech, three BiLSTM layers with 256 cells in each layer and 900 time steps were used in RNN blocks. In CNN blocks, three convolutional blocks with two convolutional layers in each block followed by batch normalization and maximum pooling layers were used (with the number of filters 16, 32, 64, kernel size 7 with stride 1, pooling size 3, and "same" padding). The dense layer outputs a prediction whether or not a frame contains a GCI. The dotted speech signals at the top indicate that no GCI was detected in the corresponding speech frames; otherwise, red circles mark GCI location.

The model is created and a summary is outputted

In [6]:
model = ut.create_model((None, n_wsamples, 1), len(filters), n_convs, filters, kernel_size, strides, padding, pool_size,
                        rnn_cells, dropout, batch_norm, rnn)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 160, 1)]    0         
_________________________________________________________________
time_distributed (TimeDistri (None, None, 160, 4)      16        
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 160, 4)      16        
_________________________________________________________________
activation (Activation)      (None, None, 160, 4)      0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 80, 4)       0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 320)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         197120

Let's now compile the model

In [7]:
opt = {'adam': Adam(learning_rate=learning_rate)}[optimizer]
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

Then, we can train the model on the train set and evaluate it on the validation set:

In [8]:
history, stopped_epoch, best_epoch_idx = ut.fit_model(model, X_train, Y_train, validation_data=(X_val, Y_val),
                                                      epochs=n_epochs, batch_size=batch_size, verbose=1)

Epoch 1/4
2/2 [==============================] - 8s 2s/step - loss: 1.0083 - accuracy: 0.4762 - val_loss: 0.6855 - val_accuracy: 0.7481
Epoch 2/4
2/2 [==============================] - 4s 1s/step - loss: 0.7870 - accuracy: 0.7107 - val_loss: 0.6675 - val_accuracy: 0.8958
Epoch 3/4
2/2 [==============================] - 4s 1s/step - loss: 0.6890 - accuracy: 0.6517 - val_loss: 0.6543 - val_accuracy: 0.8983
Epoch 4/4
2/2 [==============================] - 4s 2s/step - loss: 0.6400 - accuracy: 0.6793 - val_loss: 0.6442 - val_accuracy: 0.9025


In this very simplified example, the accuracy on the validation set was about 90% (it could differ due to the randomness). Much better results can be obtained when more training data from more voice talents is used, when tuning the hyper-parameters (such as the batch size and learning rate) and the complexity of the model (the number of CNN and/or RNN layers, the number of LSTMS cells etc.) is done and also when the model is trained for more epochs. Please see the [paper](paper/Matousek_ANNPR2022_paper.pdf) for more details.

## Evaluating the CNN-BiLSTM model

GCI detection techniques are usually evaluated by comparing locations of the detected and ground truth GCIs in terms of identification rate (IDR), miss rate (MR), false alarm rate (FAR), identification rate (IDA), accuracy within 0.25 ms range (A25), and we also use a dynamic evaluation measure (E10) (please see the [paper](paper/Matousek_ANNPR2022_paper.pdf) for more details.). 

We can evaluate our simple model and see the results

In [9]:
results = ut.evaluate_model(model, valset, spc_dir, pm_dir, freq_samp, n_timesteps, win_len, hop_len,
                            window, pad_value, sync, abs_dist, rel_dist, min_t0)
results

{'IDR': 0.14392059553349876,
 'MR': 0.8461538461538461,
 'FAR': 0.009925558312655087,
 'IDA': 0.0006717221939355704,
 'A25': 0.9516129032258065,
 'E10': 0.13647642679900746}

As can be seen, our simple model performes very poorly, with the _identification rate_ (IDR) being only 14.39%. Definitely, a better model is needed. We have tuned the hyper-parameters of the CNN-BiLSTM model and trained it on all data (3200 utterances) using GPU (see the [paper](paper/Matousek_ANNPR2022_paper.pdf)). The resulting pre-trained weigths are available [here](models/CNN-BiLSTM/weights.h5): `models/CNN-BiLSTM/weights.h5` and the model's architecture is [here](models/CNN-BiLSTM/architecture.json): `models/CNN-BiLSTM/architecture.json`. They could be used to set up the model

In [10]:
with open('models/CNN-BiLSTM/architecture.json', 'rt') as json_file:
    model = model_from_json(json_file.read())
# Load optimal model's weights from hdf5 file
model.load_weights('models/CNN-BiLSTM/weights.h5')

Now, we achieve much better results (IDR = 97.52%)

In [11]:
results = ut.evaluate_model(model, valset, spc_dir, pm_dir, freq_samp, n_timesteps, win_len, hop_len,
                            window, pad_value, sync, abs_dist, rel_dist, min_t0)
results

{'IDR': 0.9751861042183623,
 'MR': 0.009925558312655087,
 'FAR': 0.01488833746898263,
 'IDA': 0.0002446933776474915,
 'A25': 0.9924812030075187,
 'E10': 0.967741935483871}

GCI detection results for the proposed model (CNN-BiLSTM) and the comparison with other models and algorithms on publicly available [CMU](http://festvox.org/dbs/index.html) datasets are shown in the following table. Again, please see the [paper](paper/Matousek_ANNPR2022_paper.pdf) for more details.

![evaluation](figs/evaluation.png "GCI detection evaluation")

## Detecting GCIs with the CNN-BiLSTM model

For detection, we use unseen speech waveforms. Here we use three utterances from the [CMU](http://festvox.org/dbs/index.html) dataset (voice [BDL](http://festvox.org/cmu_arctic/dbs_bdl.html)).

In [12]:
predlist = ['bdl_arctic_a0001', 'bdl_arctic_a0002', 'bdl_arctic_a0003']
pms, _ = ut.detect(model, predlist, 'prediction/spc', freq_samp, n_timesteps, win_len, hop_len, window, pad_value, sync)

We can view the times of the detected GCIs, e.g. the first 10 GCIs in the first utterance:

In [13]:
[p.time for p in pms[0][:10]]

[0.2346875,
 0.2423125,
 0.25025,
 0.2580625,
 0.2659375,
 0.273875,
 0.2816875,
 0.289375,
 0.2970625,
 0.30475]

Finally, we can store the detected GCIs in the [wavesurfer](http://www.speech.kth.se/wavesurfer) format using the `Pitchmark` object

In [14]:
for pm, fn in zip(pms, predlist):
    # Write GCIs to a file
    pm.write_file(osp.join('prediction/gci_pred', fn+'.pm'))

And this is the example of the detected GCIs:

![GCI detection sample](figs/gci_detection_sample.png "GCI detection sample")